In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [18]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Carregando {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Carregando {file}')
        loader = Docx2txtLoader(file)
    else:
        print('Formato não suportado!')
        return None

    data = loader.load()
    return data

# Wikipedia Loader
def load_from_wikipedia(query, lang='pt', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [24]:
# Teste carregando dados de um pdf
# documento = load_document('docs/CLT.pdf')
# documento[100].page_content

In [25]:
# Teste carregando dados da wikipedia
# data = load_from_wikipedia('GPT-4')
# print(data[0].page_content)

In [26]:
def chunk_data(data, chunk_size=1000):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [32]:
def embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_tokens / 1000 * 0.0001:.6f}')

In [36]:
data = load_document('docs/CLT.pdf')
chunks = chunk_data(data)

Carregando docs/CLT.pdf


In [38]:
# Teste print chunks
# print(chunks[100].page_content)
# print(len(chunks))

In [39]:
# Custo total para transformar os chunks em embeddings
embedding_cost(chunks)

Total de tokens: 252697
Custo de Embedding em USD: 0.025270


In [40]:
def insert_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name in pinecone.list_indexes():
        print(f'Index {index_name}')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Criando index {index_name}')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
    return vector_store

In [41]:
def delete_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deletando todos os índices...')
        for index in indexes:
            pinecone.delete_index(index)
    else:
        print(f'Deletando índice {index_name}...')
        pinecone.delete_index(index_name)

In [44]:
# Teste deletando um índice existente
# delete_index()

In [43]:
index_name = 'linuxtips'
vector_store = insert_embeddings(index_name)

Criando index linuxtips
Ok


In [63]:
def get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(q)
    return answer

def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history

In [68]:
q = 'em que ano a clt foi criada?'
answer = get_answer(vector_store, q)
print(answer)

A CLT, Consolidação das Leis do Trabalho, foi criada em 1943.


In [48]:
import time
i = 1
print('Digite sair para encerrar.')
while True:
    q = input(f'Pergunta: #{i}: ')
    i = i+1
    if q.lower() in ['sair']:
        print('Encerrando...')
        time.sleep(2)
        break

    answer = get_answer(vector_store, q)
    print(f'\nResposta: {answer}')
    print(f'\n {"-" * 50} \n')

Digite sair para encerrar.


Pergunta: #1:  o que é o décimo terceiro salário?



Resposta: O décimo terceiro salário é um benefício pago aos trabalhadores no Brasil, também conhecido como "gratificação de Natal". Ele corresponde a um salário extra que é pago uma vez por ano, geralmente em dezembro, e equivale a 1/12 avos da remuneração total recebida pelo trabalhador no ano, incluindo salário-base, horas extras, comissões e adicionais. Esse benefício tem como objetivo proporcionar aos trabalhadores uma renda adicional para despesas extras no período de fim de ano.

 -------------------------------------------------- 



Pergunta: #2:  qual foi a minha pergunta anterior?



Resposta: Desculpe, mas eu não tenho acesso à pergunta anterior. Posso ajudar com alguma outra informação?

 -------------------------------------------------- 



Pergunta: #3:  sair


Encerrando...


In [75]:
chat_history = []
question = 'em que ano a clt foi criada?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])

A Consolidação das Leis do Trabalho (CLT) foi criada em 1943.


In [76]:
question = 'estamos em 2023, quantos anos passaram?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])

Desde 1943 até 2023, se passarão 80 anos.


In [84]:
print(chat_history)

[('em que ano a clt foi criada?', 'A Consolidação das Leis do Trabalho (CLT) foi criada em 1943.'), ('estamos em 2023, quantos anos passaram?', 'Desde 1943 até 2023, se passarão 80 anos.')]
